In [3]:
# Importation des bibliothèques  
from sentence_transformers import SentenceTransformer  
from sklearn.neighbors import NearestNeighbors  
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  
import pickle  
import os  
import warnings  
warnings.filterwarnings("ignore")  

In [4]:
# Chargement des données préparées  
print("Chargement des données préparées...")  
try:  
    # Essayer de charger depuis le fichier pickle  
    with open('../data/processed/clean_dataset.pkl', 'rb') as f:  
        df = pickle.load(f)  
    print("Données chargées depuis le fichier pickle.")  
except:  
    # Sinon, charger depuis le CSV  
    df = pd.read_csv('../data/processed/clean_dataset.csv')  
    print("Données chargées depuis le fichier CSV.")  
  
print(f"Forme du dataset: {df.shape}")  
print("\nAperçu des données:")  
display(df.head())  

Chargement des données préparées...
Données chargées depuis le fichier pickle.
Forme du dataset: (5000, 14)

Aperçu des données:


,Uniq Id,Product Id,Product Name,Product Rating,Clicked,View_Time_Sec,Product Description,Product Tags,Product Category,Product Brand,Product Reviews Count,Full_Text,Norm_View_Time,Rating
0,1705736792d82aa2f2d3caf1c07c53f4,2e17bf4acecdece67fc00f07ad62c910,"OPI Infinite Shine, Nail Lacquer Nail Polish, ...",4.218750,0,0,,"OPI Infinite Shine, Nail Lacquer Nail Polish, ...",Premium Beauty > Premium Makeup > Premium Nail...,OPI,571.035565,"opi infinite shine, nail lacquer nail polish,...",0.000000,2.109375
1,95a9fe6f4810fcfc7ff244fd06784f11,076e5854a62dd283c253d6bae415af1f,"Nice n Easy Permanent Color, 111 Natural Mediu...",3.966667,1,147,Pack of 3 Pack of 3 for the UPC: 381519000201 ...,"Nice 'n Easy Permanent Color, 111 Natural Medi...",Beauty > Hair Care > Hair Color > Auburn Hair ...,Nice'n Easy,571.035565,pack of 3 pack of 3 for the upc: 381519000201 ...,0.816667,4.300000
2,8d4d0330178d3ed181b15a4102b287f2,8a4fe5d9c7a6ed26cc44d785a454b124,Clairol Nice N Easy Permanent Color 7/106A Nat...,4.500000,0,0,This Clairol Nice N Easy Permanent Color gives...,Clairol Nice 'N Easy Permanent Color 7/106A Na...,Beauty > Hair Care > Hair Color > Permanent Ha...,Clairol,29221.000000,this clairol nice n easy permanent color gives...,0.000000,2.250000
3,fddc4df45b35efd886794b261f730c51,03b5fb878a33eadff8b033419eab9669,"Kokie Professional Matte Lipstick, Hot Berry, ...",4.024138,0,0,Calling all matte lip lovers! Indulge in our r...,"Kokie Professional Matte Lipstick, Hot Berry, ...",Beauty > Makeup > Lip,Kokie Cosmetics,571.035565,calling all matte lip lovers! indulge in our r...,0.000000,2.012069
4,0990cf89a59ca6a0460349a3e4f51d42,ce3d761e57d6ccad80619297b5b1bcbc,"Gillette TRAC II Plus Razor Blade Refills, Fit...",5.000000,1,177,"In 1971, Gillette introduced the Trac II razor...","Gillette TRAC II Plus Razor Blade Refills, Fit...",Seasonal > Stock Up Essentials > Personal Care...,Gillette,131.000000,"in 1971, gillette introduced the trac ii razor...",0.983333,4.983333


In [5]:
  
# Préparation des données pour le filtrage basé sur le contenu  
print("\n Préparation des données pour le filtrage basé sur le contenu (BERT + KNN)...")  
  
# Création d'un DataFrame avec des produits uniques  
products_df = df.drop_duplicates('Product Id')  
print(f"Nombre de produits uniques: {products_df['Product Id'].nunique()}")  


 Préparation des données pour le filtrage basé sur le contenu (BERT + KNN)...
Nombre de produits uniques: 4802


In [6]:
 
# Génération des embeddings BERT  
print("\n Génération des embeddings BERT...")  
# Chargement du modèle BERT pré-entraîné  
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  
print("Modèle BERT chargé: paraphrase-MiniLM-L6-v2")  
  
# Génération des embeddings pour chaque produit  
print("Génération des embeddings pour les descriptions de produits...")  
embeddings = model.encode(products_df['Full_Text'].tolist(), show_progress_bar=True)  
print(f"Forme des embeddings: {embeddings.shape}")  


 Génération des embeddings BERT...
Modèle BERT chargé: paraphrase-MiniLM-L6-v2
Génération des embeddings pour les descriptions de produits...


Batches:   0%|          | 0/151 [00:00<?, ?it/s]

Forme des embeddings: (4802, 384)


In [7]:
  
#  Modèle KNN pour BERT  
print("\n Entraînement du modèle KNN sur les embeddings BERT...")  
knn_model = NearestNeighbors(  
    n_neighbors=10,       
    algorithm='auto',       
    metric='cosine'       
)  
knn_model.fit(embeddings)  
print("Modèle KNN entraîné avec succès.")  


 Entraînement du modèle KNN sur les embeddings BERT...
Modèle KNN entraîné avec succès.


In [8]:
  
# Exemple de recommandation  
print("\n Exemple de recommandation avec BERT + KNN:")  
# Sélectionner un produit aléatoire  
sample_idx = np.random.randint(0, len(products_df))  
sample_product = products_df.iloc[sample_idx]  
print(f"Produit sélectionné: {sample_product['Product Name']}")  
print(f"Catégorie: {sample_product['Product Category']}")  
  
# Trouver les produits similaires  
sample_vector = embeddings[sample_idx].reshape(1, -1)  
distances, indices = knn_model.kneighbors(sample_vector)  
  
print("\nProduits similaires:")  
for i, idx in enumerate(indices[0][1:6]):   
    similar_product = products_df.iloc[idx]  
    print(f"{i+1}. {similar_product['Product Name']} (Similarité: {1-distances[0][i+1]:.4f})")  
    print(f"   Catégorie: {similar_product['Product Category']}")  


 Exemple de recommandation avec BERT + KNN:
Produit sélectionné: Hair Building Fiber Refill Bag by Finally Hair - 25 Gram Refill Bag Hair Filler Fibers Used To Thicken Hair, Fill In Balding Thinning Areas (Golden Blonde)
Catégorie: Premium Beauty > Premium Hair Care & Hair Tools > Premium Hair & Scalp Treatments

Produits similaires:
1. Joico Moisture Recovery Treatment Balm For Thick/Course Hair 8.5 Oz (Similarité: 0.6346)
   Catégorie: Premium Beauty > Premium Hair Care > Premium Hair & Scalp Treatments
2. Matrix Biolage ColorLast Wash for color treated hair 33.8 fl oz (Similarité: 0.6214)
   Catégorie: Premium Beauty > Premium Hair Care & Hair Tools > Premium Shampoos
3. Moroccanoil Strong Styling Hair Gel, 6 Oz (Similarité: 0.6195)
   Catégorie: Premium Beauty > Premium Hair Care > Premium Styling Products > Hair Styling Products
4. Ag Hair Cosmetics Ultradynamics Extra-Firm Finishing Hairspray 10 Oz (Similarité: 0.6159)
   Catégorie: Premium Beauty > Premium Hair Care & Hair Tool

In [9]:
# Évaluation  
print("\n Évaluation du modèle BERT + KNN:")  
# Calcul de la précision de la catégorie  
same_category_count = 0  
total_recommendations = 0  
  
# Échantillonner 100 produits aléatoires pour l'évaluation  
sample_size = min(100, len(products_df))  
sample_indices = np.random.choice(len(products_df), sample_size, replace=False)  
  
for idx in sample_indices:  
    product = products_df.iloc[idx]  
    product_vector = embeddings[idx].reshape(1, -1)  
    distances, indices = knn_model.kneighbors(product_vector)  
      
    # Vérifier si les 5 premiers produits recommandés sont de la même catégorie  
    for rec_idx in indices[0][1:6]:  
        recommended_product = products_df.iloc[rec_idx]  
        if recommended_product['Product Category'] == product['Product Category']:  
            same_category_count += 1  
        total_recommendations += 1  
  
category_precision = same_category_count / total_recommendations  
print(f"Précision de la catégorie: {category_precision:.4f}") 


 Évaluation du modèle BERT + KNN:
Précision de la catégorie: 0.3040


In [10]:
  
# Sauvegarde des résultats pour l'évaluation comparative  
results = {  
    'bert_knn': {  
        'category_precision': category_precision  
    }  
}  
  
#  Sauvegarde du modèle et des résultats  
print("\n Sauvegarde du modèle et des résultats...")  
  
# Créer les dossiers s'ils n'existent pas  
os.makedirs('../models/content_based', exist_ok=True)  
os.makedirs('../results', exist_ok=True)  
  
# Sauvegarde des embeddings BERT  
with open('../models/content_based/bert_embeddings.pkl', 'wb') as f:  
    pickle.dump(embeddings, f)  
print("Embeddings BERT sauvegardés dans '../models/content_based/bert_embeddings.pkl'")  
  
# Sauvegarde du modèle KNN  
with open('../models/content_based/bert_knn_model.pkl', 'wb') as f:  
    pickle.dump(knn_model, f)  
print("Modèle KNN pour BERT sauvegardé dans '../models/content_based/bert_knn_model.pkl'")  
  
# Sauvegarde des résultats  
with open('../results/bert_results.pkl', 'wb') as f:  
    pickle.dump(results, f)  
print("Résultats sauvegardés dans '../results/bert_results.pkl'")  
  
print("\n Filtrage basé sur le contenu (BERT + KNN) terminé!")


 Sauvegarde du modèle et des résultats...
Embeddings BERT sauvegardés dans '../models/content_based/bert_embeddings.pkl'
Modèle KNN pour BERT sauvegardé dans '../models/content_based/bert_knn_model.pkl'
Résultats sauvegardés dans '../results/bert_results.pkl'

 Filtrage basé sur le contenu (BERT + KNN) terminé!
